# Comparison with ChatGPT <br>

The following Code was executed in <b>Google Colab</b>, since there were some compatability-problems with local jupyter. <br>
Here, I wanted to see how a LLM would do in the task - can ChatGPT differentiate between route instructions and sights - <br>
as I did before manually - and how good does it perform in such a special NER-task? 

The code used below as well as the additional jinja-file was provided by Nina Claudia Rastinger, ÖAW. 

In [1]:
import os
import sys

!pip install git+https://github.com/promptslab/Promptify.git
from promptify import Prompter, OpenAI, Pipeline

from google.colab import drive
drive.mount('/content/drive')

text = open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/Sonntag_Teil.txt', 'r', encoding='utf8')
Sonntag_text = ' '.join(text)


In [2]:
api_key = 'your_key'

In [3]:
def prompter_ner(text):
    result = pipe.fit(text,
                      domain = "alte Skripte",
                      examples = one_shot,
                      labels = ["Ort_auf_Route", "Ort_im_Blickfeld"])
    return result#[0]["parsed"]["data"]["completion"][0]

In [ ]:
model = OpenAI(api_key)
prompter = Prompter('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/ner_adapted.jinja')
pipe = Pipeline(prompter, model)

In [ ]:
one_shot_text = "Geht man nun auf den Wall rechts (Südwest) am Palais des Erzherzogs Karl vorüber, so kömmt man zu der Auffahrt vom Spitalplatze, am Hofbau= amte vorbei, zum Kärnthnerthor=Theater, zu welchem ein bedeckter Gang vom Wall hinüber führt. Wenige Schritte weiter steht man über dem Kärnthnerthore und sieht in die Kärnthnerstraße hinab. Auf dem Glacis sieht man die Brücke über die Wien, welche zur Vorstadt Wieden, auf die Straße nach Baden führt. Rechts steht das große Starhembergische Freihaus, links das Polytech= nische Jnstitut und die Karlskirche."

In [4]:
one_shot = [[one_shot_text,
             [{"E": "Wall", "T": "Ort_auf_Route"},
              {"E": "Palais des Erzherzogs Karl", "T": "Ort_auf_Route"},
              {"E": "Spitalplatze", "T": "Ort_auf_Route"},
              {"E": "Hofbau= amte", "T": "Ort_auf_Route"},
              {"E": "Kärnthnerthor=Theater", "T": "Ort_im_Blickfeld"},
              {"E": "Kärthnerthore", "T": "Ort_auf_Route"},
              {"E": "Kärnthnerstraße", "T": "Ort_im_Blickfeld"},
              {"E": "Glacis", "T": "Ort_auf_Route"},
              {"E": "Brücke über die Wien", "T": "Ort_im_Blickfeld"},
              {"E": "Starhembergische Freihaus", "T": "Ort_im_Blickfeld"},
              {"E": "Polytech= nische Jnstitut", "T": "Ort_im_Blickfeld"},
              {"E": "Karlskirche", "T": "Ort_im_Blickfeld"}]]]

In [ ]:
part1_text = open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/Sonntag_Teil.txt', 'r', encoding='utf8') #unpickling it 
Sonntag_1 = ' '.join(part1_text)  

In [5]:
result = prompter_ner(Sonntag_1) 

I repeated the above function 8 times, using always another bit of my text (I tried to do it with the whole data at once, filesize around 13 KB, but repeatedly got Invalid request oder Rate Limit error). <br>
For really validating the performance, you probably would have to repeat the while process a couple of times, seeing how good the model performs round after round. <br>
Since this was just a mere experiment, I refrained from that. <br>
But this could be another task for the future. 

In [ ]:
#for entity in result:
  #print(entity)

In [ ]:
import pickle
import pandas as pd

In [ ]:

## this process of course has to be repeated for each single result, in case you want to save them in between steps! 
with open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/result_data.pkl', 'wb') as fp: #saving the returned object for later use
    pickle.dump(result, fp)

In [ ]:
def unpack_and_order(result):
  first = result[0]
  second = first['text']
  third = second[2:-2]

  data = eval(third)

  df = pd.DataFrame(columns=['Entity', 'Label'])

  rows = []
  for entry in data:
      if 'E' in entry and 'T' in entry:
          row = {'Entity': entry['E'], 'Label': entry['T']}
          rows.append(row)

  df_filled = pd.concat([df, pd.DataFrame(rows)])
  return df_filled

In [169]:
file1= open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/result_data.pkl', 'rb')
result_1 = pickle.load(file1, encoding='utf8')

file2 = open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/result2_data.pkl', 'rb')
result_2 = pickle.load(file2, encoding='utf8')

file3 = open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/result3_data.pkl', 'rb')
result_3 = pickle.load(file3, encoding='utf8')

file4 = open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/result4_data.pkl', 'rb')
result_4 = pickle.load(file4, encoding='utf8')

file5 = open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/result5_data.pkl', 'rb')
result_5 = pickle.load(file5, encoding='utf8')

file6 = open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/result6_data.pkl', 'rb')
result_6 = pickle.load(file6, encoding='utf8')

file7 = open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/result7_data.pkl', 'rb')
result_7 = pickle.load(file7, encoding='utf8')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/result_data.pkl'

In [ ]:
df_1 = unpack_and_order(result_1)
df_2 = unpack_and_order(result_2)
df_3 = unpack_and_order(result_3)
df_4 = unpack_and_order(result_4)
df_5 = unpack_and_order(result_5)
#df_6 = unpack_and_order(result_6)  #here, we had a /n error in our result
df_7 = unpack_and_order(result_7)

In [ ]:
#correction for sixth result
first = result_6[0]
second = first['text']
third = second[2:-2]

fourth = third.replace("\n", "")
data = eval(fourth)

df = pd.DataFrame(columns=['Entity', 'Label'])
rows = []
for entry in data:
    if 'E' in entry and 'T' in entry:
        row = {'Entity': entry['E'], 'Label': entry['T']}
        rows.append(row)

df_6 = pd.concat([df, pd.DataFrame(rows)])


In [ ]:
frames = [df_1, df_2, df_3, df_4, df_5, df_6, df_7]
full_df = pd.concat(frames)

In [ ]:
full_df.to_csv('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/full_df.csv', index=False)

Here, I got back to my local jupyter. You can find all the data & resources used in Golab in this file. <br>
<br>
Let's start by importing the full_df back into the kernel! 

In [2]:
import os 
import pandas as pd
import numpy as np


In [5]:
full_df = pd.read_csv('../ChatGPT_API/ReiseführerProjekt/full_df.csv')

In [6]:
print(full_df.head())

                                 Entity             Label
0                                  Burg  Ort_im_Blickfeld
1         Ritter= oder 6 Ceremoniensaal     Ort_auf_Route
2  Appartements Sr Majestät des Kaisers  Ort_im_Blickfeld
3        Antiken= und Mineralienkabinet     Ort_auf_Route
4                                  Burg     Ort_auf_Route


In [13]:
print(len(full_df[full_df['Label'] == 'Ort_auf_Route']))
print(len(full_df[full_df['Label'] == 'Ort_im_Blickfeld']))

40
91


Since the task for ChatGPT was to identify the places that lay on the route as well as the places that were visible at the several parts of the route, I assigned two labels: place on a route, and place in sight. <br> 
At first glance, looking only at the numbers, the model didn't perform so bad: It found 178 places, 50 of them on the route, and 128 of them in sight - which seems to be a pretty sensible ratio!  
<br>
<br>
Now, we should see if we can match the places the model found with the places I manually extracted from the text, and at the same time add the coordinates we already found on our map. 

In [14]:
# import the dataset for sunday with all places & coordinates

coordinate_table = pd.read_excel('C:/Users/sarah/Desktop/git_folder/DataScienceProject/Data/Sonntag/joined_Sonntag_coord_nn.xlsx')

In [47]:
print(coordinate_table.head())

   Index                     NEUE_NAMEN              ADRESSE Sicherheit  \
0      0                     k. k. Burg                  NaN       hoch   
1      1                      Burgplatz                  NaN       hoch   
2      2                       Burgthor                  NaN       hoch   
3      3        kaiserlichen Stallungen                  NaN       hoch   
4      4  Gebäude der ungarischen Garde  07., Museumstraße 7     mittel   

   Longitude   Latitude                             Quelle  \
0  16.365809  48.206874  Gazetteer_Steinhausenplan_V5_auto   
1  16.364500  48.206794                 QGIS - Estimation    
2  16.362702  48.205781                 QGIS - Estimation    
3  16.358862  48.203593                 QGIS - Estimation    
4  16.355278  48.205833                 Wikipedia, GeoHack   

                                                 URL  
0                                                NaN  
1                                                NaN  
2          

In [15]:
# extract the first column as a list to do some fuzzy string matching

place_list = coordinate_table['NEUE_NAMEN'].tolist()

In [16]:
from thefuzz import process, fuzz 

In [17]:
full_df['MATCH_SCORE'] = full_df['Entity'].apply(lambda x: process.extractOne(x, place_list, scorer=fuzz.ratio)[1])
full_df['NEUE_NAMEN'] = full_df['Entity'].apply(lambda x: process.extractOne(x, place_list, scorer=fuzz.ratio)[0])
full_df = full_df.sort_values('MATCH_SCORE', ascending=False)

In [18]:
full_df.head(50)

,Entity,Label,MATCH_SCORE,NEUE_NAMEN
0,Burg,Ort_im_Blickfeld,100,Burg
76,Kahlengebirge,Ort_im_Blickfeld,100,Kahlengebirge
89,Erdberg,Ort_im_Blickfeld,100,Erdberg
86,Bastei,Ort_auf_Route,100,. Bastei
85,Franzensbrücke,Ort_im_Blickfeld,100,Franzensbrücke
84,Kornspeicher,Ort_im_Blickfeld,100,Kornspeicher
83,Donau,Ort_im_Blickfeld,100,Donau
81,Jägerzeile,Ort_im_Blickfeld,100,Jägerzeile
80,Taborstraße,Ort_auf_Route,100,Taborstraße
79,Ferdinandsbrücke,Ort_auf_Route,100,Ferdinandsbrücke


In [19]:
full_df.tail(50)

,Entity,Label,MATCH_SCORE,NEUE_NAMEN
92,Poli= zeihaus,Ort_im_Blickfeld,92,Polizeihaus
18,Hofbau= amte,Ort_auf_Route,91,Hofbauamte
90,Rothen= thurmthore,Ort_auf_Route,91,Rothenthurm=Thore
3,Antiken= und Mineralienkabinet,Ort_auf_Route,89,AntikenMineralienkabinet
23,Antiken= und Mineralienkabinet,Ort_im_Blickfeld,89,AntikenMineralienkabinet
26,Burg= gartens,Ort_auf_Route,87,Burggarten
9,Burg= gartens,Ort_im_Blickfeld,87,Burggarten
121,große Kavallerie=Kaserne,Ort_im_Blickfeld,86,Kavallerie=Kaserne
24,Burg= platze,Ort_auf_Route,86,Burgplatz
7,Burg= platze,Ort_auf_Route,86,Burgplatz


A reasonable limit would be 61, I propose - after that, there's not much to be matched correctly. <br>
Single exeptions like Hauptmauth can be added manually later on. <br> But from the look of it, only around 20 seem to be kind of wrong, some definitely also the fault of the textsource the model had to work with. 


In [20]:
full_df.loc[full_df.MATCH_SCORE.between(0,60), 'NEUE_NAMEN'] = np.nan

In [21]:
print(full_df)

                                               Entity             Label  \
0                                                Burg  Ort_im_Blickfeld   
76                                      Kahlengebirge  Ort_im_Blickfeld   
89                                            Erdberg  Ort_im_Blickfeld   
86                                             Bastei     Ort_auf_Route   
85                                     Franzensbrücke  Ort_im_Blickfeld   
..                                                ...               ...   
35                               Brücke über die Wien  Ort_im_Blickfeld   
44  Kaserne (für Jnfanterie und das Beschäl=Depart...  Ort_im_Blickfeld   
62                 Bezirk LandstraßeVorstadt = Bezirk     Ort_auf_Route   
14  Reiterstatue des Gemahls von M. Theresia, Kais...  Ort_im_Blickfeld   
29                                        Flügelsalon  Ort_im_Blickfeld   

    MATCH_SCORE      NEUE_NAMEN  
0           100            Burg  
76          100   Kahlengebirge

In [22]:
full_df.sort_index(axis=0, ascending=True, inplace=True) #recreate the correct order

In [23]:
full_df

,Entity,Label,MATCH_SCORE,NEUE_NAMEN
0,Burg,Ort_im_Blickfeld,100,Burg
1,Ritter= oder 6 Ceremoniensaal,Ort_auf_Route,82,RitterCeremoniensaal
2,Appartements Sr Majestät des Kaisers,Ort_im_Blickfeld,100,Appartements Sr Majestät des Kaisers
3,Antiken= und Mineralienkabinet,Ort_auf_Route,89,AntikenMineralienkabinet
4,Burg,Ort_auf_Route,100,Burg
...,...,...,...,...
126,Volksgarten,Ort_im_Blickfeld,100,Volksgarten
127,Schenkenstraße,Ort_im_Blickfeld,100,Schenkenstraße
128,Palais Liech= tenstein,Ort_im_Blickfeld,95,Palais Liechtenstein
129,doppelte Brücke,Ort_im_Blickfeld,62,große Steinbrücke


Now, let's combine both dataframes depending on the NEUE_NAMEN column in order to create our QGIS dataframe.

In [24]:
result = pd.merge(full_df, coordinate_table, on="NEUE_NAMEN", how="left")
result = result.drop(columns=["MATCH_SCORE", "Index"])

In [25]:
result.to_excel('C:/Users/sarah/Desktop/git_folder/DataScienceProject/Data/Sonntag/Chatgpt_coord.xlsx')

In [27]:
## I had to delete some rows that were duplicated & manually added some missing coordinates
result_new = pd.read_excel('C:/Users/sarah/Desktop/git_folder/DataScienceProject/Data/Sonntag/Chatgpt_coord_nn.xlsx')
result_new.to_csv('C:/Users/sarah/Desktop/git_folder/DataScienceProject/Data/Sonntag/Chatgpt_coord_nn.csv')

To compare the performance of ChatGPT with the manual reconstruction, following steps were taken: 
<br>
1. Calculate Accuracy: <br> Since the ChatGPT model contained many duplicates (e.g., four times count of "Burg" directly behind each other), <br>
I decided to calculate accuracy using only the unique entries. Of course, some places occur more than once also in the original route. <br>
To keep the balance, only unique entries for the model as well as the manual selection were chosen: <br>
In comparison to the manual, ChatGPT was therefore able to identify 92 correct places out of 159 unique entries. 
The rest of ChatGPTs found places (in total, the model found 178) were either duplicates or not included in the manual selection. 
<br>
<br>
2. Calculate Precision, Recall and F1-Score: <br>
ChatGPT identified 11 places correctly as route points (TP) and falsely identified 14 sights as route points (FP). <br> This was counted manually comparing both dataframes as well as the map, because the manual reconstruction dataframe did not differentiate between route - & sight locations. Duplicates were not taken into account, except for the places that were correctly identitied as both lying on the route as well as being sights. <br>
For locations in view, ChatGPT identified 61 places correctly (TN) and misidentified 9 places as route points. (FN). <br>
<br> 
For both categories, Precision, Recall and F1-Score were calculated manually: <br>
<br>
<b>Route-Points</b>: 
- Precision: TP/ TP+FP = 11/11+14 = 0,44
- Recall: TP/ TP+FN = 11/11+9 = 0,55
- F1-Score: 2*Precision * Recall/ Precision + Recall = 0,484/0,99 = 0,48888889
<br><br>
<b>Sights</b>:
- Precision: 61/61+9 = ~0,8714
- Recall: 61/61+14 = ~0,8133
- F1-Score: 1,417/ 1,685 = ~0,8409
